<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 1
## <center> Анализ данных по сердечно-сосудистым заболеваниям
*Идеальный просмотр Jupyter notebooks возможен только локально, GitHub и nbviewer могут неправильно отбражать формулы или картинки.*

**В задании предлагается с помощью Pandas ответить на несколько вопросов по данным о сердечно-сосудистых заболеваниях (качать данные не надо, они уже есть в репозитории). Данные использовались в соревновании [Ml Boot Camp 5](http://mlbootcamp.ru/round/12/sandbox/).**

**Заполните код в клетках (где написано "Ваш код здесь") и ответьте на вопросы в [веб-форме](https://docs.google.com/forms/d/1KbWiGlupiSwYlqc8nmV6KjB3vNTI0C5hxZ6Tq3ZU4QA).**

В соревновании предлагалось определить наличие/отсутствие сердечно-сосудистых заболеваний (ССЗ) по результатам осмотра пациента.

**Описание данных.**

Датасет сформирован из реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

Объективные признаки:

 - Возраст (age)
 - Рост (height)
 - Вес (weight)
 - Пол (gender)
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее (ap_hi, ap_lo)
 - Холестерин (cholesterol)
 - Глюкоза (gluc)
 

Субъективные признаки (со слов пациентов):

 - Курение (smoke)
 - Употребление алкоголя (alco)
 - Физическая активность (active)
 
Целевой признак (который интересно будет прогнозировать):
 - Наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра (cardio)

Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Мы будем работать только с обучающей выборкой и с помощью `Pandas` проведем первичный анализ данных. 

Из библиотек нам понадобятся только `NumPy` и `Pandas`.

In [2]:
import numpy as np
import pandas as pd

Считываем данные из CSV-файла в объект pandas DataFrame.  

In [3]:
df = pd.read_csv('../../data/mlbootcamp5_train.csv', sep=';', 
                 index_col='id')

Посмотрим на первые 5 записей.

In [4]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


**<font color='red'>Вопрос 1 (1 балл).</font> Сколько мужчин и женщин представлено в этом наборе данных? Не было дано расшифровки признака "пол" (какому полу соответствует 1, а какому – 2 в признаке ``gender``) – это определите, посмотрев также на рост при разумном предположении, что в среднем мужчины выше (здесь и далее под средним понимается среднее арифметическое).**

**<font color='red'>Варианты:</font>**
- 45470 женщин и 24530 мужчин
- 45470 мужчин и 24530 женщин
- 45530 женщин и 24470 мужчин
- 45530 мужчин и 24470 женщин

In [25]:
height_1 = df[df['gender'] == 1]['height'].mean()
height_2 = df[df['gender'] == 2]['height'].mean()

print("Средний рост пола №1: {}".format(height_1))
print("Средний рост пола №2: {}".format(height_2))
# Предположим, что женщины ниже мужчин:
if (height_1 > height_2):
    female_gender = 2
    male_gender = 1
else: 
    female_gender = 1
    male_gender = 2
print("Примем мужской пол = {}, женский = {}".format(male_gender, female_gender))

male_count = df[df['gender'] == male_gender]['gender'].count()
female_count = df[df['gender'] == female_gender]['gender'].count()
print("Мужчин: {}\nЖенщин: {}".format(male_count, female_count))

Средний рост пола №1: 161.35561168460356
Средний рост пола №2: 169.94789538210054
Примем мужской пол = 2, женский = 1
Мужчин: 24470
Женщин: 45530


**<font color='red'>Вопрос 2 (1 балл).</font> Кто в среднем реже указывает, что употребляет алкоголь – мужчины или женщины?**

**<font color='red'>Варианты:</font>**
- мужчины 
- женщины

In [33]:
df_males = df[df['gender'] == male_gender]
df_females = df[df['gender'] == female_gender]

males_alcohol = df_males['alco'].mean()
females_alcohol = df_females['alco'].mean()

print("Мужчины пьют: {}\nЖенщины пьют: {}".format(males_alcohol, females_alcohol))
if (males_alcohol > females_alcohol):
    print("Мужчины пьют чаще")
else:
    print("Женщины пьют чаще")

Мужчины пьют: 0.10637515324887617
Женщины пьют: 0.02549967054689216
Мужчины пьют чаще


**<font color='red'>Вопрос 3 (1 балл).</font> Во сколько раз (округленно, `round`) процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?**

**<font color='red'>Варианты:</font>**
- 4
- 8
- 12
- 16

In [35]:
males_smoke = df_males['smoke'].mean()
females_smoke = df_females['smoke'].mean()

print("Мужчины курят: {}\nЖенщины курят: {}".format(males_smoke, females_smoke))
print("Мужчины курят в {} раз чаще женщин.".format(round(males_smoke / females_smoke)))

Мужчины курят: 0.21888026154474868
Женщины курят: 0.017856358444981332
Мужчины курят в 12 раз чаще женщин.


**<font color='red'>Вопрос 4 (1 балл).</font> Вы наверняка заметили, что значения возраста какие-то странные. Догадайтесь, в чём здесь измеряется возраст, и ответьте, на сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих.**

**<font color='red'>Варианты:</font>**
- 10
- 15
- 20
- 25

In [52]:
df['age_years'] = df['age'] / 365.25

nonsmoker_median_age = df[df['smoke'] == False]['age_years'].median()
smoker_median_age = df[df['smoke'] == True]['age_years'].median()
print("Медианный возраст нормального человека: {} лет".format(nonsmoker_median_age))
print("Медианный возраст курильщика: {} лет".format(smoker_median_age))
print("Разница в месяцах: {}".format((nonsmoker_median_age - smoker_median_age) * 12))

Медианный возраст нормального человека: 53.9958932238193 лет
Медианный возраст курильщика: 52.36139630390144 лет
Разница в месяцах: 19.61396303901438


**<font color='red'>Вопрос 5 (2 балла).</font> В [статье](https://ru.wikipedia.org/wiki/Сердечно-сосудистый_риск) на Википедии про сердечно-сосудистый риск показана шкала SCORE для расчёта риска смерти от сердечно-сосудистого заболевания в ближайшие 10 лет. Вот она:**
<img src='https://habrastorage.org/webt/gg/38/vf/gg38vfl5ql8obqjmlsvt9es76p8.png' width=70%>

Давайте посмотрим на правый верхний прямоугольник, отображающий сегмент курящих мужчин в возрасте от 60 до 64 лет включительно. (Неочевидно, но тут для возраста и давления цифры означают верхнюю границу, и она не включается).

Видим 9-ку в левом нижнем углу этого прямоугольника и 47 – в правом верхнем. То есть если при этом систолическое (т.е. верхнее) артериальное давление – меньше 120 мм рт.ст., а уровень холестерина – 4 ммоль/л, то риск ССЗ оценивается примерно в 5 раз ниже, чем если бы давление лежало в интервале [160, 180), а холестерина было бы 8 ммоль/л.

Давайте посчитаем аналогичное значение, но на наших данных. 

Уточнения:
- Создайте новый признак ``age_years`` – возраст в годах, округлив до целых (`round`). Для данного примера отберите курящих мужчин от 60 до 64 лет включительно
- Категории уровня холестрина на рисунке и в наших данных отличаются. Отображение значений на картинке в значения признака ``cholesterol`` следующее: 4 ммоль/л $\rightarrow$ 1, 5-7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.
- Интересуют 2 подвыборки курящих мужчин возраста от 60 до 64 лет включительно: первая с верхним артериальным давлением строго меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л, а вторая – с верхним артериальным давлением от 160 (включительно) до 180 мм рт.ст. (не включительно) и концентрацией холестерина – 8 ммоль/л.

Во сколько раз (округленно, `round`) отличаются доли больных людей (согласно целевому признаку, `cardio`) в этих двух подвыборках? Посчитайте на наших данных.

**<font color='red'>Варианты:</font>**
- 2
- 3
- 4
- 5

In [70]:
df['age_years'] = (round(df['age'] / 365.25)).astype('int')

df_certain_males = df[(df['gender'] == male_gender) & (df['age_years'] >= 60) & (df['age_years'] <= 64)]

df_certain_males_low_pressure = df[(df['ap_hi'] < 120) & (df['cholesterol'] == 1)]
df_certain_males_high_pressure = df[(df['ap_hi'] >= 160) & (df['ap_hi'] < 180) & (df['cholesterol'] == 3)]

low_pressure_cardio_chance = df_certain_males_low_pressure['cardio'].mean()
high_pressure_cardio_chance = df_certain_males_high_pressure['cardio'].mean()

print("low_pressure_low_cholesterol: {}".format(low_pressure_cardio_chance))
print("high_pressure_high_cholesterol: {}".format(high_pressure_cardio_chance))
print("differ by: {}".format(round(high_pressure_cardio_chance / low_pressure_cardio_chance)))

low_pressure_low_cholesterol: 0.2040739343643908
high_pressure_high_cholesterol: 0.8667582417582418
differ by: 4


**<font color='red'>Вопрос 6 (2 балла).</font> Постройте новый признак – BMI ([Body Mass Index](https://en.wikipedia.org/wiki/Body_mass_index)). Для этого надо вес в килограммах поделить на квадрат роста в метрах. Нормальными считаются значения BMI от 18.5 до 25. Выберите верные утверждения.**

**<font color='red'>Утверждения:</font>**
- Медианный BMI по выборке превышает норму
- У женщин в среднем BMI ниже, чем у мужчин
- У здоровых в среднем BMI выше, чем у больных
- В сегменте здоровых и непьющих мужчин в среднем BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [93]:
df['bmi'] = df['weight'] / (df['height'] * df['height']) * 100 * 100
print("Медианный индекс массы тела: {}".format(df['bmi'].median()))
print("BMI женщин: {}, BMI мужчин: {}".format(df[df['gender'] == female_gender]['bmi'].mean(), df[df['gender'] == male_gender]['bmi'].mean()))
print("BMI здоровых: {}, BMI больных: {}".format(df[df['cardio'] == False]['bmi'].mean(), df[df['cardio'] == True]['bmi'].mean()))
print("BMI одних: {}, BMI других: {}".format(df[(df['cardio'] == False) & (df['alco'] == False) & (df['gender'] == male_gender)]['bmi'].mean(), df[(df['cardio'] == False) & (df['alco'] == False) & (df['gender'] == female_gender)]['bmi'].mean()))

print("\ntrue\nfalse\nfalse\ntrue")

Медианный индекс массы тела: 26.37406812077498
BMI женщин: 27.987583441833113, BMI мужчин: 26.75444235728947
BMI здоровых: 26.548175206794745, BMI больных: 28.566060627015435
BMI одних: 25.872638075460173, BMI других: 26.84540659413151

true
false
false
true


**<font color='red'>Вопрос 7 (2 балла).</font> ** Можно заметить, что данные не особо-то чистые, много в них всякой "грязи" и неточностей. Еще лучше мы это увидим, когда обсудим визуализацию данных.

Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)
- указанное нижнее значение артериального давления строго выше верхнего
- рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили (используйте `pd.Series.quantile`, если не знаете, что это такое – прочитайте)
- вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

Это вовсе не вся чистка данных, которую можно было проделать, но пока остановимся на этом.

Сколько процентов данных (округленно, `round`) мы выбросили?

**<font color='red'>Варианты:</font>**
- 8
- 9
- 10
- 11

In [124]:
height_lower_threshold = df['height'].quantile(2.5 * 0.01)
height_upper_threshold = df['height'].quantile(97.5 * 0.01)
weight_lower_threshold = df['weight'].quantile(2.5 * 0.01)
weight_upper_threshold = df['weight'].quantile(97.5 * 0.01)

df_remains = df[df['ap_lo'] <= df['ap_hi']]
df_remains = df_remains[(df_remains['height'] >= height_lower_threshold) & (df_remains['height'] <= height_upper_threshold)]
df_remains = df_remains[(df_remains['weight'] >= weight_lower_threshold) & (df_remains['weight'] <= weight_upper_threshold)]

print("Было: {}".format(df.shape[0]))
print("Осталось: {}".format(df_remains.shape[0]))
print("Выбросили: {}({}%)".format(df.shape[0] - df_remains.shape[0], round((df.shape[0] - df_remains.shape[0]) / df.shape[0]  * 100)))

Было: 70000
Осталось: 63259
Выбросили: 6741(10%)
